In [162]:
import math
import time
from random import randrange

class player:
    def __init__(self, name_, pool_size, num =-1, pieces=0, score=0) -> None:    
        self.name=name_

        if num != -1:
            self.number=num #generated indep so multiple people can have the same number
        else:
            self.number = randrange(1,100)
        
        self.cards = ["+","x","/", "z"]
        self.pieces =pieces
        self.score = score
        self.guesses = [None] * pool_size


In [122]:
def csvline2list(line): #input processing
    print(line)
    acc=""
    end_list = []
    for c in line:
        if (c==","):
            end_list+=[acc]
            acc=""
        elif(c==" "):
            pass
        else:
            acc = acc+c
    end_list+=[acc]
    return end_list


In [123]:
def handle_op(op, n1, n2): #BSN LOGIC
    if (op == "+"):
        result = n1 + n2 #both nums should already be ints
        if (result <20):
            return "between 3-20"
        elif (result>180):
            return "between 180-198"
        else:
            return str(result)
    elif (op=="x"):
        return str(n1*n2)[-1]
    elif(op=="/"):
        return str(math.floor(max(abs(n1/n2), abs(n2/n1))))
    elif(op=="z"):
        v1 = math.floor(n1/10)
        v2 = math.floor(n2/10)
        return str(max(v1-v2, v2-v1))
    else:
        print("Error, invalid operation")
        return "non-existent move"

In [124]:
#TESTING
#result = csvline2list("alice, bob, colin,    daphne, eloise") //unit test csvline2list

# hey = handle_op("/", 8, 93)
# print("unit test: divide operator")
# if (hey == '11'): print("PASSED")
# else: print("FAILED")

hey = handle_op("z", 9, 93)
print("unit test: zero operator")
print(hey)
if (hey == '9'): print("PASSED")
else: print("FAILED")

unit test: zero operator
9
PASSED


In [165]:
def init_players(players): #BSN LOGIC
    #proc input
    p_input= input("Enter a comma separated list of all participants: ") #does not check for duplicates
    p_names = csvline2list(p_input)

    #init players
    p_num = len(p_names)

    for p in p_names:
        players+= [player(p, p_num)]

    #assign pieces
    for r in range(math.floor(p_num/2)): #a third of the players get pieces
        players[randrange(p_num)].pieces= players[randrange(p_num)].pieces+1 #done w replacement so worst case 1 person could get all

    return players

In [150]:
def test_init_players(players): #TESTING
    print("Test init")
    players += [player("alice",3, 93, 2),
                player("bob", 3, 44, 0),
                player("liza", 3, 91, 3)]

    return players

In [153]:
def print_players(pp):#Utility
    for p in pp:
        attributes = vars(p)
        print(', '.join("%s: %s" % item for item in attributes.items()))

In [166]:
#TESTING

yeet = []
init_players(yeet) 
print_players(yeet)


a,b,c,d,e,f
name: a, number: 90, cards: ['+', 'x', '/', 'z'], pieces: 0, score: 0, guesses: [None, None, None, None, None, None]
name: b, number: 29, cards: ['+', 'x', '/', 'z'], pieces: 0, score: 0, guesses: [None, None, None, None, None, None]
name: c, number: 67, cards: ['+', 'x', '/', 'z'], pieces: 1, score: 0, guesses: [None, None, None, None, None, None]
name: d, number: 37, cards: ['+', 'x', '/', 'z'], pieces: 0, score: 0, guesses: [None, None, None, None, None, None]
name: e, number: 77, cards: ['+', 'x', '/', 'z'], pieces: 1, score: 0, guesses: [None, None, None, None, None, None]
name: f, number: 68, cards: ['+', 'x', '/', 'z'], pieces: 0, score: 0, guesses: [None, None, None, None, None, None]


In [129]:
def proc_play(id1, id2, op, players): #ids are int #BSN LOGIC
#needs testing of list modification

    try:
        p1 = players[id1]
        p2 = players[id2]

        if(p1 is p2): print("Error, cannot check operand card with yourself")
            
    except(IndexError):
        print("Error, Unknown player id. Please retry your request.")
        return

    if (op in p1.cards and op in p2.cards):
        print("your result is " + handle_op(op, p1.number, p2.number))
        p1.cards.remove(op)
        p2.cards.remove(op)
    else:
        print("Error, at least one card is missing.")
    
    return

In [145]:
def handle_stdin():
    print("Welcome to numguesser!")

    players = []
    test_init_players(players)
    #init_players(players)

    #explain rules TODO

    #set timer #nothing under this has been tested at all
    start = time.time()
    limit = 5#60*10 #ten minutes in seconds

    #main loop
    while(True):

        instr = input( "Enter your request (play, buy, hand, check or end)")
        instr = instr.strip()

        if (instr =="play"):
            op = input("Enter the operand card you would like to play:")
            p1 = input("Enter player 1 id:")
            p2 = input ("Enter player 2 id:")
            
            proc_play(int(p1.strip()), int(p2.strip()),op.strip(), players)
        
        elif (instr=="buy"):
            p1 = input("Enter player id:")
        
            #start proc_buy

            try:
                p1 = players[int(p1.strip())]
            except(IndexError):
                print("Error, Unknown player id. Please retry your request.")
                continue
            
            if (p1.pieces>0): 
                p1.pieces = p1.pieces-1
                p1.cards += ["+","x","/","z"]
                print("you have successfully bought another set of 4 cards, new piece count is " + str(p1.pieces))
            else:
                print("Error, you do not have enough pieces.")
            #end proc_buy
        
        elif(instr =="hand"):
            p1 = input("Enter player id:")
            
            try:
                p1 = players[int(p1.strip())]
            except(IndexError):
                print("Error, Unknown player id. Please retry your request.")
                continue

            print("Your current hand is ", p1.cards)

        elif (instr=="check"): #partly for debugging
            print_players(players)

        elif(instr=="end"):
            break

        else:
            print("Error, unknown command. Please try again.")



        #check self pieces, check self cards, search id by name    

        #instruction to see remaining cards/view check history (add logging to file)

        #allow making guesses

    #at game end, check values
    #calculate scores



# if __name__ == "__main__":
#     main()

In [146]:
handle_stdin()

Welcome to numguesser!
Test init
name: alice, number: 93, cards: ['+', 'x', '/', 'z'], pieces: 2, score: 0, guesses: {}
name: bob, number: 44, cards: ['+', 'x', '/', 'z'], pieces: 0, score: 0, guesses: {}
name: liza, number: 91, cards: ['+', 'x', '/', 'z'], pieces: 3, score: 0, guesses: {}


In [135]:
def simple_timer():
    start = time.time()
    limit = 5
    while (True):
        if time.time() > start+limit:
            print("woop")
            break